In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('../raw_data/Clean_data_1_12_v_model.csv')
data=data.drop(columns='Unnamed: 0')
data=data.head(100)

In [3]:
X = data.drop(columns=['rating','description','name', 'address', 'label', 'postal_code', 'type', 'municipality', 'review_count'])
y = data['rating']

In [4]:
X.head(3)

,price,dine_in,takeaway,delivery,drive_through,no_del_exp,curb_pickup,neighborhood
0,2.0,1,1,0,0,1,0,Graça
1,2.0,1,1,1,0,0,0,Prazeres
2,1.0,1,1,0,0,1,0,Santa Engrácia


In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [14]:
price_transformer = SimpleImputer(strategy="most_frequent")
cat_transformer = OneHotEncoder()

preproc_basic = make_column_transformer((price_transformer, ['price']),
                                       (cat_transformer, ['neighborhood'])
                                       #(cat_transformer, ['neighborhood', 'type'])
                                        , remainder='passthrough')

pipe = make_pipeline(preproc_basic, Ridge())
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['neighborhood'])])),
                ('ridge', Ridge())])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [21]:
pipe.fit(X_train,y_train)
pipe.score(X_test,y_test)

AttributeError: This 'Pipeline' has no attribute 'transform'

In [18]:
pipe.get_params()

grid_search = GridSearchCV(
    pipe, 
    param_grid={
        'ridge__alpha': [0.1, 0.5, 1, 5, 10]},
    cv=5,
    scoring="r2")

grid_search.fit(X_train, y_train)
grid_search.best_params_

/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/met

/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/louisecaucheteux/.pyenv/versions/3.8.12/envs/small_business/lib/python3.8/site-packages/sklearn/met

{'ridge__alpha': 0.1}

In [19]:
pipe.predict(X_test.iloc[0:2])

array([4.66451505, 4.45160122])

In [20]:
X_test.iloc[0:2]

,price,dine_in,takeaway,delivery,drive_through,no_del_exp,curb_pickup,neighborhood
80,2.0,1,0,1,0,0,0,Encarnação
84,2.0,1,1,1,0,0,0,Coração de Jesus
